In [38]:
import torch 
import glob
import random
import os
from torch.utils.data import DataLoader
import torchvision.transforms as transforms
import sys

In [58]:
import torchvision
from torch.utils.data import Dataset
from ipywidgets import interact, widgets
import matplotlib.pyplot as plt 
from PIL import Image
%matplotlib inline


In [59]:
from models import *
from data import *

In [60]:
loadSize = 143
fineSize = 128
ngf = 64
ndf = 64    
in_nc = 3 
out_nc = 3 
cuda=False

##### Initialize Generator

In [61]:
gen = Generator(in_nc,out_nc,4,ngf)

In [62]:
gen.load_state_dict(torch.load('./netG4.pth',map_location='cpu'))

IncompatibleKeys(missing_keys=[], unexpected_keys=[])

###### Define Image Transforms

In [63]:
transforms_ = [ transforms.Resize(int(128), Image.BICUBIC), 
        transforms.RandomCrop(128), 
        #transforms.RandomVerticalFlip(p=0),
        transforms.ToTensor(),
        transforms.Normalize((0.5,0.5,0.5), (0.5,0.5,0.5)) 
      ]

transform = transforms.Compose(transforms_)

In [64]:
def tensor2image(tensor):
    image = 127.5*(tensor[0].cpu().float().detach().numpy() + 1.0)
    return image.astype(np.uint8)

# ###### Testing#####

In [66]:
@interact(
          vango=widgets.FloatSlider(min=0,max=1,step=.01,value=0,continuous_update=False)
          ,ukyo=widgets.FloatSlider(min=0,max=1,step=.01,value=0,continuous_update=False)
          ,monet=widgets.FloatSlider(min=0,max=1,step=.01,value=0,continuous_update=False)
          ,cezan=widgets.FloatSlider(min=0,max=1,step=.01,value=0,continuous_update=False)
          ,ident=widgets.FloatSlider(min=.01,max=1,step=.01,value=1,continuous_update=False)
)
def generate_image(vango,ukyo,monet,cezan,ident,
                path='./58103064467__9C25377E-2D03-488E-9EA5-3F768533DA6E.JPG',
                flip90=False):
    generator=gen
    content = transform(Image.open(path))
    Tensor = torch.cuda.FloatTensor if cuda else torch.Tensor
    input_A = Tensor(1, 3, 128, 128)
    real_A = input_A.copy_(content)
    #print(style_label)
    
    generated = gen({
        'content': real_A,
        'style_label': [[cezan,monet,ukyo,vango,ident]]   
    })
    im=tensor2image(generated.data)
    if flip90:
        im=im.transpose(2,1,0)
    else:
        im=im.transpose(1,2,0)
    fig = plt.figure(figsize=(6,6))
    ax = fig.add_subplot(111)  
    ax.imshow(im)



interactive(children=(FloatSlider(value=0.0, continuous_update=False, description='vango', max=1.0, step=0.01)…